In [13]:
from pycbc.waveform import get_td_waveform
import h5py
from pycbc.filter.matchedfilter import match
import pycbc
import pycbc.psd
import numpy as np

In [14]:
# Generate a time-domain waveform
hp, hc = get_td_waveform (approximant = 'SEOBNRv4',
                         mass1 = 20,
                         mass2 = 20,
                         eccentricity = 0.,
                         long_asc_nodes = 0.,
                         inclination = 0.,
                         f_lower = 30.,
                         delta_t = 1.0/4096)

def GetMatch (template0,template1, psd_file ='H1L1-O1_C02_HARM_MEAN_PSD-1126051217-11203200.txt', f_low=30):
    #resize two templates
    tlen = max(len(template0),len(template1))
    template1.resize(tlen)
    template0.resize(tlen)

    #grab and use the psd file
    my_psd = pycbc.psd.read.from_txt(filename = psd_file,
                                    length = tlen,
                                    delta_f = template0.delta_f,
                                    low_freq_cutoff = f_low,
                                    is_asd_file = False)

    #calculate match
    m,i = match(template0,template1,psd=my_psd,low_frequency_cutoff=f_low)
    return m

def SingleFF(comp_mass1, comp_mass2, waveform0, tp_apx, tp_m1, tp_m2, tp_ecc, tp_lan, tp_inc, radius, f_low=30):
    waveform0_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(comp_mass1, comp_mass2)
    matches = []
    for k in range(0, len(tp_m1)):
        template_m1 = tp_m1[k]
        template_m2 = tp_m2[k]
        template_mchirp = pycbc.conversions.mchirp_from_mass1_mass2(template_m1, template_m2)
        diff = abs(waveform0_mchirp - template_mchirp)
        percentage_diff = diff/waveform0_mchirp
        if (percentage_diff > radius):
            matches.append(0.)
        else: 
            sp, sc = get_td_waveform(approximant = 'EccentricTD',
                                    mass1 = tp_m1[k],
                                    mass2 = tp_m2[k],
                                    eccentricity = tp_ecc[k],
                                    long_asc_nodes = tp_lan[k],
                                    inclination = tp_inc[k],
                                    f_lower = f_low,
                                    delta_t = waveform0.delta_t)
            this_match = GetMatch(waveform0, sp)
            matches.append(this_match)
    matches = np.asarray(matches)
    ff = np.amax(matches)
    ff_index = int(np.argmax(matches))
    best_match_m1 = tp_m1[ff_index]
    best_match_m2 = tp_m2[ff_index]
    return ff, best_match_m1, best_match_m2


In [16]:
# Grab the template bank info
f_bank = h5py.File('bank0729_atlas1_new6.hdf', 'r')
bank_mass1 = f_bank['mass1'][:]
bank_mass2 = f_bank['mass2'][:]
bank_ecc = f_bank['eccentricity'][:]
bank_lan = f_bank['long_asc_nodes'][:]
bank_inc = f_bank['inclination'][:]
bank_apx = f_bank['approximant'][:]
f_bank.close()

print (len(bank_mass1))
print (np.amax(bank_mass1))
print (np.amin(bank_mass2))

353288
49.99998137567024
5.0000019363985295


In [ ]:
ff, best_match_m1, best_match_m2 = SingleFF(comp_mass1 = 20.,
                                                    comp_mass2 = 20.,
                                                    waveform0 = hp,
                                                    tp_apx = bank_apx, 
                                                    tp_m1 = bank_mass1,
                                                    tp_m2 = bank_mass2,
                                                    tp_ecc = bank_ecc,
                                                    tp_lan = bank_lan,
                                                    tp_inc = bank_inc,
                                                    radius = 0.01,
                                                    f_low = 30.)
print ff
print best_match_m1
print best_match_m2